In [1]:
import pandas as pd
import shapefile
import os
from json import dumps
import folium
import numpy as np
from IPython.display import IFrame

In [2]:
fname = './data/tl_2010_06037_tabblock10/tl_2010_06037_tabblock10'

In [3]:
def inBox(lat, lon):
    #if lat < 33.8 or lat > 34.25 or lon < -118.7 or lon > -118.0: #big
    if lat < 34.00 or lat > 34.06 or lon < -118.30 or lon > -118.25: #small
    #if lat < 34.00 or lat > 34.1 or lon < -118.30 or lon > -118.2: #small2
        return False
    return True

In [12]:
def generateJobs():
    jobs_20_pd = pd.read_csv('./data/net_am_peak/jobs_am_all_20.csv',dtype={'0':int})
    jobs_20_pd.rename(columns={'0':'job_20'}, inplace=True)
    jobs_20_pd = jobs_20_pd.drop_duplicates(subset='node_id',keep='last')
    jobs_40_pd = pd.read_csv('./data/net_am_peak/jobs_am_all_40.csv', dtype={'0':int})
    jobs_40_pd.rename(columns={'0':'job_40'}, inplace=True)
    jobs_40_pd = jobs_40_pd.drop_duplicates(subset='node_id',keep='last')
    jobs_60_pd = pd.read_csv('./data/net_am_peak/jobs_am_all_60.csv',dtype={'0':int})
    jobs_60_pd.rename(columns={'0':'job_60'}, inplace=True)
    jobs_60_pd = jobs_60_pd.drop_duplicates(subset='node_id',keep='last')
    jobs_pd = jobs_20_pd[['node_id', 'job_20']].merge(jobs_40_pd[['node_id','job_40']], on='node_id',how='inner')
    jobs_pd = jobs_pd.merge(jobs_60_pd[['node_id','job_60']], on='node_id')
    census_block_db = pd.read_csv('./data/net_am_peak/block_am_closest_node.csv',dtype={'GEOID10':str})
    jobs_pd2 = jobs_pd.merge(census_block_db[['node_id','GEOID10','INTPTLAT10','INTPTLON10']], on='node_id',how='right')
    jobs_pd2.set_index('GEOID10', inplace=True)
    return jobs_pd2

In [13]:
jobs_pd2 = generateJobs()

In [26]:
def shape2json(fname, outfile="./data/blocks.json", country='Brazil'):
    reader = shapefile.Reader(fname)
    fields = reader.fields[1:]
    field_names = [field[0] for field in fields]

    data = []
    for sr in reader.shapeRecords():
        atr = dict(zip(field_names, sr.record))
        geom = sr.shape.__geo_interface__
        lat = float(atr['INTPTLAT10'])
        lon = float(atr['INTPTLON10'])
        if inBox(lat, lon):
            jobs = jobs_pd2.loc[atr['GEOID10']]
            jobs_dict = {"id":atr['GEOID10'],"min20":int(jobs['job_20']),"min40":int(jobs['job_40']), "min60":int(jobs['job_60'])}
            data.append(dict(type="Feature", geometry=geom, properties=jobs_dict))
            #data.append(dict(type="Feature", geometry=geom))

    with open(outfile, "w") as geojson:
        geojson.write(dumps({"type": "FeatureCollection",
                             "features": data}, indent=2) + "\n")

In [27]:
shape2json(fname)

In [16]:
#lat < 33.8 or lat > 34.25 or lon < -118.7 or lon > -118.0
bbox = [-118.7, 33.8, -118.0, 34.25]

lon_center, lat_center = np.array(bbox).reshape(2, 2).mean(axis=0)
mapa = folium.Map(#width=650, height=500,
                  zoom_start=12,
                  #tiles='Mapbox Bright',
                  location=[lat_center, lon_center])

mapa.geo_json(geo_path='./data/blocks.json', fill_color='D46A6A', line_color='White')

#inline_map(mapa)

In [17]:
for i in range(0, len(jobs_pd2)):
    row = jobs_pd2.iloc[i]
    lat = row['INTPTLAT10']
    lon = row['INTPTLON10']
    if inBox(lat, lon):
        pop_info = 'GEOID10: ' + row['GEOID10'] + '\n Number of jobs reachable in\n 20 minutes: ' + str(row['job_20'])\
        + '\n 40 minutes: ' + str(row['job_40']) + '\n 60 minutes: ' + str(row['job_60'])
        folium.Marker([lat, lon], popup=pop_info).add_to(mapa)

In [9]:
jobs_pd2 = jobs_pd2.reset_index('GEOID10')

In [18]:
mapa.save('./data/interactive2.html')

In [17]:
row = jobs_pd2.loc['060379800241037']
int(row['job_20'])

134608

In [18]:
jobs_pd2

,node_id,job_20,job_40,job_60,INTPTLAT10,INTPTLON10
GEOID10,,,,,,
060379800241037,14,134608,662345,1811906,34.171673,-118.500541
060373111003013,36,262504,1122007,2309108,34.178659,-118.356128
060371242012011,41,216970,1019407,2217107,34.180328,-118.374405
060371241022001,44,234769,1189294,2289660,34.175334,-118.381584
060371241022008,50,230246,1168148,2283495,34.174480,-118.379723
060371239012004,55,200557,1088425,2209900,34.180317,-118.391594
060371239013030,56,200557,1088300,2209898,34.181303,-118.394103
060371239022004,56,200557,1088300,2209898,34.178282,-118.394604
060371236022007,64,153466,1032642,2070065,34.180409,-118.427381
